In [61]:
import cirq
from mitiq import zne, pec, benchmarks

In [65]:
circuit = benchmarks.generate_rb_circuits(n_qubits=2, num_cliffords=50)[0]

print(circuit)

0: ───Y^0.5────X^0.5───Y^0.5───@───Y^0.5────@───Y^0.5───Y^-0.5───X^-0.5───Y^0.5──────────@───X^0──────Y────────X^0.5────────────────────────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5────X^0.5───Y^0───@───X^0──────Y^0.5────X^0─────────────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───Y^0.5───X^0.5───@───X^0───Y^-0.5───X^0.5───Y^-0.5───@───Y^-0.5───@───Y^0.5────@───Y───────X───────────────────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───Y^0.5───X^0.5───@───Y^0.5───X^0.5───Y^0.5────X^0───@───Y^0.5────@───Y───────X^0.5───X^-0.5───Y^0───@───Y^0.5───X^0.5───X───────Y^0.5───────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5────Y───X───@───Y^0.5────@───Y^0.5───X^0───@───X^0─────Y────────X^0.5───@───Y^0.5────@───Y───────X^0.5───Y───────X─────────────@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───Y^0.5────X^0───@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───X───Y^0.5───@───X^0───X^-0.5───Y^0───────────@───X^-0.5───Y^-0.5───X─────Y^0.5───@───Y^0.5────@───X^-0.5───Y^-0.5───X^0.5───Y^0.5───X^0───@───Y^0

In [63]:
def execute(circuit, noise_level=0.001):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit with depolarizing noise."""
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

In [66]:
true_value = execute(circuit, noise_level=0.0)      # Ideal quantum computer.
noisy_value = execute(circuit)                      # Noisy quantum computer.
zne_value = zne.execute_with_zne(circuit, execute)  # Noisy quantum computer + Mitiq.

print(f"Error (w/o  Mitiq): %0.4f" % abs((true_value - noisy_value) / true_value))
print(f"Error (with Mitiq): %0.4f" % abs((true_value - zne_value) / true_value))

Error (w/o  Mitiq): 0.3799
Error (with Mitiq): 0.0509


In [67]:
qubit = cirq.LineQubit(0)
circ = cirq.Circuit([cirq.H(qubit)])

print(circ)

0: ───H───


In [68]:
rep = pec.represent_operation_with_local_depolarizing_noise(
    circ,
    noise_level=0.01,
)

print(rep)

0: ───H─── = 1.010*(0: ───H───)-0.003*(0: ───H───X───)-0.003*(0: ───H───Y───)-0.003*(0: ───H───Z───)


In [69]:
pec_val = pec.execute_with_pec(
    circ,
    executor=execute,
    representations=[rep],
    num_samples=50,
)

print("Mitigated result:", round(pec_val, 3))

Mitigated result: 0.51
